In [1]:
import pandas as pd

In [2]:
merged_dataframe_n1 = pd.read_csv("local.eu.effects.tsv",sep="\t", header=0)

In [3]:
merged_dataframe_n1.columns

Index(['molecular_trait_id', 'chromosome_x', 'position', 'ref', 'alt',
       'variant', 'ma_samples', 'maf', 'pvalue', 'beta', 'se', 'type', 'ac',
       'an', 'r2', 'molecular_trait_object_id', 'gene_id', 'median_tpm',
       'rsid', 'position0', 'chromosome_y', 'gene_start', 'gene_end',
       'phenotype_id'],
      dtype='object')

In [4]:
merged_dataframe_n1.shape

(843015, 24)

In [5]:
merged_dataframe_n2 = merged_dataframe_n1[['chromosome_x', 'position0', 'position', 'ref', 'alt', 'molecular_trait_id',
       'variant', 'ma_samples', 'maf', 'pvalue', 'beta', 'se', 'type', 'ac',
       'an', 'r2', 'molecular_trait_object_id', 'gene_id', 'median_tpm',
       'rsid',  'gene_start', 'gene_end',
       'phenotype_id']]

In [6]:
merged_dataframe_n2['chromosome_x'] = "chr"+merged_dataframe_n2['chromosome_x'].astype(str)

/users/anusri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
merged_dataframe_n2.shape

(843015, 23)

In [9]:
import pybedtools
peaks_data_merged=pd.read_csv("/mnt/lab_data2/anusri/kumasaka/new_download/Kumasaka_2018_wgs/sumstats/merged.peaks.eu.top.50k.bed", sep="\t", header=None)
print(peaks_data_merged.head())
beds_var = pybedtools.BedTool.from_dataframe(merged_dataframe_n2)
peak_bed = pybedtools.BedTool.from_dataframe(peaks_data_merged) # this is a merged peaks file
vars_in_bed = beds_var.intersect(peak_bed, c=True)

      0       1       2
0  chr1   10024   10432
1  chr1   29074   29455
2  chr1  181106  181306
3  chr1  183652  184006
4  chr1  186816  187016


In [10]:
vars_in_bedn = vars_in_bed.to_dataframe(names=merged_dataframe_n2.columns.tolist() + ["count"])


In [11]:
vars_in_bedn2 = vars_in_bedn[vars_in_bedn["count"]>0]

In [12]:
sum(vars_in_bedn2["pvalue"]<1e-7)

8385

In [13]:
sum(vars_in_bedn2["pvalue"]>1e-2)

100211

In [14]:
vars_in_bedn2.shape

(130270, 24)

In [15]:
vars_in_bedn2 = vars_in_bedn2.drop_duplicates()

In [17]:
vars_in_bedn2.shape

(130270, 24)

In [24]:
vars_in_bedn2.head(5)

,chromosome_x,position0,position,ref,alt,molecular_trait_id,variant,ma_samples,maf,pvalue,...,an,r2,molecular_trait_object_id,gene_id,median_tpm,rsid,gene_start,gene_end,phenotype_id,count
1,chr1,29367,29368,G,A,Interval_2,chr1_29368_G_A,7,0.038462,0.162300,...,182,.,Interval_2,Interval_2,.,rs796745295,28924,29435,Interval_2,1
2,chr1,181112,181113,A,G,Interval_3,chr1_181113_A_G,75,0.412088,0.008885,...,182,.,Interval_3,Interval_3,.,rs1383295402,180749,181528,Interval_3,1
16,chr1,183667,183668,G,C,Interval_4,chr1_183668_G_C,9,0.049451,0.677535,...,182,.,Interval_4,Interval_4,.,rs1375358516,182645,184537,Interval_4,1
17,chr1,183855,183856,A,G,Interval_4,chr1_183856_A_G,7,0.038462,0.990356,...,182,.,Interval_4,Interval_4,.,rs1260198881,182645,184537,Interval_4,1
21,chr1,186834,186835,A,G,Interval_5,chr1_186835_A_G,77,0.423077,0.025945,...,182,.,Interval_5,Interval_5,.,rs1364805962,186367,187110,Interval_5,1


In [25]:
#vars_in_bedn2[["chromosome_x", "position", "ref", "alt", "variant"]].to_csv("kumsaka.2018.kaur.scores.peak.filtered.bed", sep='\t', header=False, index=False)



In [26]:
len(set(vars_in_bedn2["variant"]))

130268

In [58]:
atac_df = pd.read_csv("/mnt/lab_data2/anusri/variant-scorer/src/output/kaur_caqtls_lcl_latest/ATAC/summary.mean.variant_scores_new.tsv", sep="\t", header=0)
dnase_df = pd.read_csv("/mnt/lab_data2/anusri/variant-scorer/src/output/kaur_caqtls_lcl_latest/DNASE_NEW/summary.mean.variant_scores_new_3.tsv", sep="\t", header=0)
atac_df_50m = pd.read_csv("/mnt/lab_data2/anusri/variant-scorer/src/output/kaur_caqtls_lcl_latest/ATAC_50M/summary.mean.variant_scores_new.tsv", sep="\t", header=0)



In [59]:
atac_df.shape

(130270, 23)

In [60]:
atac_df_50m.shape

(130270, 23)

In [61]:
dnase_df.shape

(130270, 24)

In [62]:
atac_df.columns

Index(['chr', 'pos', 'allele1', 'allele2', 'rsid', 'logfc.mean',
       'abs_logfc.mean', 'abs_logfc.mean.pval', 'jsd.mean', 'jsd.mean.pval',
       'logfc_x_jsd.mean', 'abs_logfc_x_jsd.mean', 'abs_logfc_x_jsd.mean.pval',
       'max_percentile.mean', 'max_percentile.mean.pval',
       'logfc_x_max_percentile.mean', 'abs_logfc_x_max_percentile.mean',
       'abs_logfc_x_max_percentile.mean.pval', 'jsd_x_max_percentile.mean',
       'jsd_x_max_percentile.mean.pval',
       'abs_logfc_x_jsd_x_max_percentile.mean',
       'abs_logfc_x_jsd_x_max_percentile.mean.pval',
       'logfc_x_jsd_x_max_percentile.mean'],
      dtype='object')

In [64]:
merged_df = atac_df[['chr', 'pos', 'allele1', 'allele2', 'rsid']]

In [65]:
merged_df["ATAC.572M.logfc.mean"] = atac_df['logfc.mean']
merged_df["ATAC.572M.jsd.mean"] = atac_df['jsd.mean']
merged_df["ATAC.572M.CRS"] = atac_df['abs_logfc_x_jsd_x_max_percentile.mean']
merged_df["ATAC.50M.logfc.mean"] = atac_df_50m['logfc.mean']
merged_df["ATAC.50M.jsd.mean"] = atac_df_50m['jsd.mean']
merged_df["ATAC.50M.CRS"] = atac_df_50m['abs_logfc_x_jsd_x_max_percentile.mean']
merged_df["DNASE.logfc.mean"] = dnase_df['logfc.mean']
merged_df["DNASE.jsd.mean"] = dnase_df['jsd.mean']
merged_df["DNASE.CRS"] = dnase_df['abs_logfc_x_jsd_x_max_percentile.mean']




/users/anusri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/users/anusri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/users/anusri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [66]:
import pickle as pkl
import os
import numpy as np

mean_change=[]
rsids=[]

enf_dir="/mnt/lab_data2/anusri/variant-scorer/src/output/kaur_caqtls_lcl_latest/enformer_preds_small_window/splitss/"
for diri in ["splitaa", "splitab", "splitac", "splitad"]:
    enformer_recompute = pkl.load(open(enf_dir+diri+"predictions_at_snp.pkl",'rb'))
    for i in range(enformer_recompute["ref_logcount_preds"].shape[0]):
        mean_change.append(np.log2( (enformer_recompute["ref_logcount_preds"][i,440:456].sum()+1) / (enformer_recompute["alt_logcount_preds"][i,440:456].sum()+1)))
        rsids.append(enformer_recompute["rsids"][i])
mean_change = np.array(mean_change)


In [67]:
merged_df["enformer_recomputed"] = mean_change

/users/anusri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [68]:
len(mean_change)

130270

In [71]:
newdf = vars_in_bedn2.reset_index()
merged_df["pvalue"] =newdf["pvalue"]
merged_df["beta"] =newdf["beta"]

/users/anusri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/users/anusri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [73]:

enf_preds = pd.read_csv("/mnt/lab_data2/anusri/enformer/kaur_eu_caqtls/enformer_predictions.tsv", sep="\t", header=0)

In [74]:
enf_preds.head(1)

,chr,pos,ref,alt,rsid,DNASE_SAD,DNASE_SAR
0,chr1,740285,G,A,rs193160839,-0.002522,-0.003365


In [76]:
enf_preds.shape

(126606, 7)

In [78]:
enf_preds.columns


Index(['chr', 'pos', 'ref', 'alt', 'rsid', 'DNASE_SAD', 'DNASE_SAR'], dtype='object')

In [79]:
merged_df.columns

Index(['chr', 'pos', 'allele1', 'allele2', 'rsid', 'ATAC.572M.logfc.mean',
       'ATAC.572M.jsd.mean', 'ATAC.572M.CRS', 'ATAC.50M.logfc.mean',
       'ATAC.50M.jsd.mean', 'ATAC.50M.CRS', 'DNASE.logfc.mean',
       'DNASE.jsd.mean', 'DNASE.CRS', 'enformer_recomputed', 'pvalue', 'beta'],
      dtype='object')

In [91]:
merged_df["dbsnp_rsdi"] = vars_in_bedn2.reset_index()["rsid"]
#merged_df

/users/anusri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [103]:
temp1 = merged_df[['chr', 'pos', 'allele2', 'allele1', 'dbsnp_rsdi']].sort_values('dbsnp_rsdi')
temp1[temp1["dbsnp_rsdi"]=="rs193160839"]

,chr,pos,allele2,allele1,dbsnp_rsdi
10,chr1,804905,A,G,rs193160839


In [101]:
temp1 = enf_preds[['chr', 'pos', 'ref', 'alt', 'rsid']]
temp1[temp1["rsid"]=="rs193160839"]

,chr,pos,ref,alt,rsid
0,chr1,740285,G,A,rs193160839


In [115]:
merged_df1 = merged_df.merge(enf_preds, right_on=['chr', 'ref', 'alt', 'rsid'], left_on=['chr', 'allele2', 'allele1', 'dbsnp_rsdi'], how="inner")
merged_df1["DNASE_SAD"] = merged_df1["DNASE_SAD"]*-1
merged_df1["DNASE_SAR"] = merged_df1["DNASE_SAR"]*-1
merged_df2 = merged_df.merge(enf_preds, right_on=['chr', 'ref', 'alt', 'rsid'], left_on=['chr', 'allele1', 'allele2', 'dbsnp_rsdi'], how="inner")

merged_df_f = pd.concat([merged_df1, merged_df2]).drop_duplicates(['chr', 'pos_x', 'allele1', 'allele2', 'rsid_x'])


In [117]:
merged_df_f.columns

Index(['chr', 'pos_x', 'allele1', 'allele2', 'rsid_x', 'ATAC.572M.logfc.mean',
       'ATAC.572M.jsd.mean', 'ATAC.572M.CRS', 'ATAC.50M.logfc.mean',
       'ATAC.50M.jsd.mean', 'ATAC.50M.CRS', 'DNASE.logfc.mean',
       'DNASE.jsd.mean', 'DNASE.CRS', 'enformer_recomputed', 'pvalue', 'beta',
       'dbsnp_rsdi', 'pos_y', 'ref', 'alt', 'rsid_y', 'DNASE_SAD',
       'DNASE_SAR'],
      dtype='object')

In [118]:
merged_df_f.shape

(125682, 24)

In [119]:
merged_df_f.to_csv("kaur.caqtls.tsv", sep="\t", header=True, index=False)

In [50]:
vars_in_bedn2.shape

(130270, 24)

In [52]:
vars_in_bedn2.columns

Index(['chromosome_x', 'position0', 'position', 'ref', 'alt',
       'molecular_trait_id', 'variant', 'ma_samples', 'maf', 'pvalue', 'beta',
       'se', 'type', 'ac', 'an', 'r2', 'molecular_trait_object_id', 'gene_id',
       'median_tpm', 'rsid', 'gene_start', 'gene_end', 'phenotype_id',
       'count'],
      dtype='object')

In [53]:
vars_in_bedn2.head(1)

,chromosome_x,position0,position,ref,alt,molecular_trait_id,variant,ma_samples,maf,pvalue,...,an,r2,molecular_trait_object_id,gene_id,median_tpm,rsid,gene_start,gene_end,phenotype_id,count
1,chr1,29367,29368,G,A,Interval_2,chr1_29368_G_A,7,0.038462,0.1623,...,182,.,Interval_2,Interval_2,.,rs796745295,28924,29435,Interval_2,1


In [54]:
#vars_in_bedn2[['chromosome_x', 'position', 'ref', 'alt', 'variant', 'rsid']].to_csv("/mnt/lab_data2/anusri/enformer/kaur_eu_caqtls/rsids.txt", sep='\t', header=False, index=False)
                                                                   